In [52]:
import pandas as pd
from pybaseball import statcast
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor

In [3]:
"""
data20_22 = statcast(start_dt = "2020-04-01", end_dt = "2022-12-01") 
data20_22.to_csv('Statcast_2020_to_2022.csv')
data23 = statcast(start_dt = "2023-04-01", end_dt = "2023-12-01")
data23.to_csv('Statcast_2023.csv')
"""

This is a large query, it may take a moment to complete
Skipping offseason dates
Skipping offseason dates
Skipping offseason dates
Skipping offseason dates


100%|██████████| 589/589 [01:54<00:00,  5.15it/s]
/Users/maxwassarman/miniconda3/envs/baseball/lib/python3.12/site-packages/pybaseball/statcast.py:85: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_data = pd.concat(dataframe_list, axis=0).convert_dtypes(convert_string=False)


This is a large query, it may take a moment to complete
Skipping offseason dates


100%|██████████| 229/229 [00:47<00:00,  4.80it/s]
/Users/maxwassarman/miniconda3/envs/baseball/lib/python3.12/site-packages/pybaseball/statcast.py:85: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_data = pd.concat(dataframe_list, axis=0).convert_dtypes(convert_string=False)


In [59]:
#Read in data
data20_22 = pd.read_csv("Statcast_2020_to_2022.csv")
data23 = pd.read_csv("Statcast_2023.csv")

In [60]:
#Select columns
data20_22 = data20_22[['player_name','pitch_type','release_speed','release_pos_x','release_pos_z','release_spin_rate','release_extension','spin_axis','pfx_x', 'pfx_z','vx0','vy0','ax','ay','az','game_year','delta_run_exp']]
data23 = data23[['player_name','pitch_type','release_speed','release_pos_x','release_pos_z','release_spin_rate','release_extension','spin_axis','pfx_x', 'pfx_z','vx0','vy0','ax','ay','az','game_year','delta_run_exp']]

In [61]:
#Drop na values
data20_22 = data20_22.dropna()
data23 = data23.dropna()

In [62]:
#Flip to get view from pitcher
data20_22['pfx_x'] = data20_22['pfx_x'] * -1
data23['pfx_x'] = data23['pfx_x'] * -1

#Change to inches
data20_22['pfx_x'] = data20_22['pfx_x'] * 12
data20_22['pfx_z'] = data20_22['pfx_z'] * 12
data23['pfx_x'] = data23['pfx_x'] * 12
data23['pfx_z'] = data23['pfx_z'] * 12

In [64]:
#Remove unnecessary pitches
data20_22 = data20_22[data20_22.pitch_type != 'PO']
data23 = data23[data23.pitch_type != 'PO']

In [65]:
#Retag pitches
data20_22['pitch_type'] = data20_22['pitch_type'].replace(['FF','FA'], 'Fastball')
data23['pitch_type'] = data23['pitch_type'].replace(['FF','FA'], 'Fastball')

data20_22['pitch_type'] = data20_22['pitch_type'].replace(['SI'], 'Sinker')
data23['pitch_type'] = data23['pitch_type'].replace(['SI'], 'Sinker')

data20_22['pitch_type'] = data20_22['pitch_type'].replace(['FC'], 'Cutter')
data23['pitch_type'] = data23['pitch_type'].replace(['FC'], 'Cutter')

data20_22['pitch_type'] = data20_22['pitch_type'].replace(['SL'], 'Slider')
data23['pitch_type'] = data23['pitch_type'].replace(['SL'], 'Slider')

data20_22['pitch_type'] = data20_22['pitch_type'].replace(['ST'], 'Sweeper')
data23['pitch_type'] = data23['pitch_type'].replace(['ST'], 'Sweeper')

data20_22['pitch_type'] = data20_22['pitch_type'].replace(['CU','CS','SV','KC'], 'Curveball')
data23['pitch_type'] = data23['pitch_type'].replace(['CU','CS','SV','KC'], 'Curveball')

data20_22['pitch_type'] = data20_22['pitch_type'].replace(['CH','FO','EP','KN','SC'], 'Changeup')
data23['pitch_type'] = data23['pitch_type'].replace(['CH','FO','EP','KN','SC'], 'Changeup')

data20_22['pitch_type'] = data20_22['pitch_type'].replace(['FS'], 'Splitter')
data23['pitch_type'] = data23['pitch_type'].replace(['FS'], 'Splitter')



In [66]:
#Seperate pitches for different models
fastball_20_22 = data20_22[data20_22.pitch_type.isin(['Fastball'])]
fastball_23 = data23[data23.pitch_type.isin(['Fastball'])]

sinker_20_22 = data20_22[data20_22.pitch_type.isin(['Sinker'])]
sinker_23 = data23[data23.pitch_type.isin(['Sinker'])]

cutter_20_22 = data20_22[data20_22.pitch_type.isin(['Cutter'])]
cutter_23 = data23[data23.pitch_type.isin(['Cutter'])]

slider_20_22 = data20_22[data20_22.pitch_type.isin(['Slider'])]
slider_23 = data23[data23.pitch_type.isin(['Slider'])]

sweeper_20_22 = data20_22[data20_22.pitch_type.isin(['Sweeper'])]
sweeper_23 = data23[data23.pitch_type.isin(['Sweeper'])]

curveball_20_22 = data20_22[data20_22.pitch_type.isin(['Curveball'])]
curveball_23 = data23[data23.pitch_type.isin(['Curveball'])]

changeup_20_22 = data20_22[data20_22.pitch_type.isin(['Changeup'])]
changeup_23 = data23[data23.pitch_type.isin(['Changeup'])]

splitter_20_22 = data20_22[data20_22.pitch_type.isin(['Splitter'])]
splitter_23 = data23[data23.pitch_type.isin(['Splitter'])]


In [67]:
#Split into train and test
fastball_x = fastball_20_22[['release_speed','release_pos_x','release_pos_z','release_spin_rate','release_extension','spin_axis','pfx_x', 'pfx_z','vx0','vy0','ax','ay','az','game_year']]
fastball_y = fastball_20_22['delta_run_exp']

x_train_fastball, x_test_fastball, y_train_fastball, y_test_fastball = train_test_split(fastball_x, fastball_y, train_size = 0.75, random_state = 12345)

sinker_x = sinker_20_22[['release_speed','release_pos_x','release_pos_z','release_spin_rate','release_extension','spin_axis','pfx_x', 'pfx_z','vx0','vy0','ax','ay','az','game_year']]
sinker_y = sinker_20_22['delta_run_exp']

x_train_sinker, x_test_sinker, y_train_sinker, y_test_sinker = train_test_split(sinker_x, sinker_y, train_size=0.75, random_state=12345)

cutter_x = cutter_20_22[['release_speed','release_pos_x','release_pos_z','release_spin_rate','release_extension','spin_axis','pfx_x', 'pfx_z','vx0','vy0','ax','ay','az','game_year']]
cutter_y = cutter_20_22['delta_run_exp']

x_train_cutter, x_test_cutter, y_train_cutter, y_test_cutter = train_test_split(cutter_x, cutter_y, train_size=0.75, random_state=12345)

slider_x = slider_20_22[['release_speed','release_pos_x','release_pos_z','release_spin_rate','release_extension','spin_axis','pfx_x', 'pfx_z','vx0','vy0','ax','ay','az','game_year']]
slider_y = slider_20_22['delta_run_exp']

x_train_slider, x_test_slider, y_train_slider, y_test_slider = train_test_split(slider_x, slider_y, train_size=0.75, random_state=12345)

sweeper_x = sweeper_20_22[['release_speed','release_pos_x','release_pos_z','release_spin_rate','release_extension','spin_axis','pfx_x', 'pfx_z','vx0','vy0','ax','ay','az','game_year']]
sweeper_y = sweeper_20_22['delta_run_exp']

x_train_sweeper, x_test_sweeper, y_train_sweeper, y_test_sweeper = train_test_split(sweeper_x, sweeper_y, train_size=0.75, random_state=12345)

curveball_x = curveball_20_22[['release_speed','release_pos_x','release_pos_z','release_spin_rate','release_extension','spin_axis','pfx_x', 'pfx_z','vx0','vy0','ax','ay','az','game_year']]
curveball_y = curveball_20_22['delta_run_exp']

x_train_curveball, x_test_curveball, y_train_curveball, y_test_curveball = train_test_split(curveball_x, curveball_y, train_size=0.75, random_state=12345)

changeup_x = changeup_20_22[['release_speed','release_pos_x','release_pos_z','release_spin_rate','release_extension','spin_axis','pfx_x', 'pfx_z','vx0','vy0','ax','ay','az','game_year']]
changeup_y = changeup_20_22['delta_run_exp']

x_train_changeup, x_test_changeup, y_train_changeup, y_test_changeup = train_test_split(changeup_x, changeup_y, train_size=0.75, random_state=12345)

splitter_x = splitter_20_22[['release_speed','release_pos_x','release_pos_z','release_spin_rate','release_extension','spin_axis','pfx_x', 'pfx_z','vx0','vy0','ax','ay','az','game_year']]
splitter_y = splitter_20_22['delta_run_exp']

x_train_splitter, x_test_splitter, y_train_splitter, y_test_splitter = train_test_split(splitter_x, splitter_y, train_size=0.75, random_state=12345)

In [68]:
#Set params for catboost models
params = {
    'iterations': 100,
    'depth': 5,
    'learning_rate': 0.01,
    'random_seed': 12345
}

In [69]:
#Train each model

#Fastball
fastball_model = CatBoostRegressor(**params)
fastball_model.fit(x_train_fastball, y_train_fastball)

#Sinker
sinker_model = CatBoostRegressor(**params)
sinker_model.fit(x_train_sinker, y_train_sinker)

#Cutter
cutter_model = CatBoostRegressor(**params)
cutter_model.fit(x_train_cutter, y_train_cutter)

#Slider
slider_model = CatBoostRegressor(**params)
cutter_model.fit(x_train_cutter, y_train_cutter)

#Sweeper
sweeper_model = CatBoostRegressor(**params)
sweeper_model.fit(x_train_sweeper, y_train_sweeper)

#Curveball
curveball_model = CatBoostRegressor(**params)
sweeper_model.fit(x_train_curveball, y_train_curveball)

#Changeup
changeup_model = CatBoostRegressor(**params)
changeup_model.fit(x_train_changeup, y_train_changeup)

#Splitter
splitter_model = CatBoostRegressor(**params)
splitter_model.fit(x_train_splitter, y_train_splitter)

0:	learn: 0.2404991	total: 14.8ms	remaining: 1.47s
1:	learn: 0.2404934	total: 26.6ms	remaining: 1.3s
2:	learn: 0.2404871	total: 37.5ms	remaining: 1.21s
3:	learn: 0.2404809	total: 49.5ms	remaining: 1.19s
4:	learn: 0.2404757	total: 60.6ms	remaining: 1.15s
5:	learn: 0.2404704	total: 73ms	remaining: 1.14s
6:	learn: 0.2404651	total: 85.8ms	remaining: 1.14s
7:	learn: 0.2404594	total: 99.5ms	remaining: 1.14s
8:	learn: 0.2404545	total: 112ms	remaining: 1.13s
9:	learn: 0.2404488	total: 124ms	remaining: 1.11s
10:	learn: 0.2404435	total: 135ms	remaining: 1.09s
11:	learn: 0.2404384	total: 149ms	remaining: 1.09s
12:	learn: 0.2404330	total: 161ms	remaining: 1.07s
13:	learn: 0.2404284	total: 172ms	remaining: 1.06s
14:	learn: 0.2404242	total: 185ms	remaining: 1.04s
15:	learn: 0.2404192	total: 198ms	remaining: 1.04s
16:	learn: 0.2404147	total: 211ms	remaining: 1.03s
17:	learn: 0.2404099	total: 224ms	remaining: 1.02s
18:	learn: 0.2404053	total: 238ms	remaining: 1.01s
19:	learn: 0.2404003	total: 250ms	re

In [81]:
#Predictions
fastball_23['pred'] = fastball_model.predict(fastball_23[['release_speed','release_pos_x','release_pos_z','release_spin_rate','release_extension','spin_axis','pfx_x', 'pfx_z','vx0','vy0','ax','ay','az','game_year']])

sinker_23['pred'] = sinker_model.predict(sinker_23[['release_speed','release_pos_x','release_pos_z','release_spin_rate','release_extension','spin_axis','pfx_x', 'pfx_z','vx0','vy0','ax','ay','az','game_year']])

cutter_23['pred'] = cutter_model.predict(cutter_23[['release_speed','release_pos_x','release_pos_z','release_spin_rate','release_extension','spin_axis','pfx_x', 'pfx_z','vx0','vy0','ax','ay','az','game_year']])

slider_23['pred'] = slider_model.predict(slider_23[['release_speed','release_pos_x','release_pos_z','release_spin_rate','release_extension','spin_axis','pfx_x', 'pfx_z','vx0','vy0','ax','ay','az','game_year']])

sweeper_23['pred'] = sweeper_model.predict(sweeper_23[['release_speed','release_pos_x','release_pos_z','release_spin_rate','release_extension','spin_axis','pfx_x', 'pfx_z','vx0','vy0','ax','ay','az','game_year']])

curveball_23['pred'] = curveball_model.predict(curveball_23[['release_speed','release_pos_x','release_pos_z','release_spin_rate','release_extension','spin_axis','pfx_x', 'pfx_z','vx0','vy0','ax','ay','az','game_year']])

changeup_23['pred'] = changeup_model.predict(changeup_23[['release_speed','release_pos_x','release_pos_z','release_spin_rate','release_extension','spin_axis','pfx_x', 'pfx_z','vx0','vy0','ax','ay','az','game_year']])

splitter_23['pred'] = splitter_model.predict(splitter_23[['release_speed','release_pos_x','release_pos_z','release_spin_rate','release_extension','spin_axis','pfx_x', 'pfx_z','vx0','vy0','ax','ay','az','game_year']])

/var/folders/8n/cvy2zw5j013f0b4bj6srsd880000gn/T/ipykernel_28707/311633319.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fastball_23['pred'] = fastball_model.predict(fastball_23[['release_speed','release_pos_x','release_pos_z','release_spin_rate','release_extension','spin_axis','pfx_x', 'pfx_z','vx0','vy0','ax','ay','az','game_year']])


In [82]:
def scale_and_score(df):
    df['scaled'] = df['pred'] - df['pred'].max()
    df['scaled'] = abs(df['scaled'])
    df['scaled'] = df['scaled'] - df['scaled'].mean()
    df['scaled'] = df['scaled'] / df['scaled'].std()
    df['score'] = df['scaled'] * 10 + 100
    return df

/var/folders/8n/cvy2zw5j013f0b4bj6srsd880000gn/T/ipykernel_28707/3619178600.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fastball_23['scaled'] = fastball_23['pred'] - fastball_23['pred'].max()
/var/folders/8n/cvy2zw5j013f0b4bj6srsd880000gn/T/ipykernel_28707/3619178600.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fastball_23['scaled'] = abs(fastball_23['scaled'])
/var/folders/8n/cvy2zw5j013f0b4bj6srsd880000gn/T/ipykernel_28707/3619178600.py:4: SettingWithCopyWarning: 
A value is trying to be se

In [83]:
def scale_and_score(df):
    df['scaled'] = df['pred'] - df['pred'].max()
    df['scaled'] = abs(df['scaled'])
    df['scaled'] = df['scaled'] - df['scaled'].mean()
    df['scaled'] = df['scaled'] / df['scaled'].std()
    df['score'] = df['scaled'] * 10 + 100
    return df

,player_name,pitch_type,release_speed,release_pos_x,release_pos_z,release_spin_rate,release_extension,spin_axis,pfx_x,pfx_z,vx0,vy0,ax,ay,az,game_year,delta_run_exp,pred,scaled,score
1,"Sborz, Josh",Fastball,96.6,-0.69,6.24,2442.0,6.1,206.0,4.20,17.40,4.913600,-140.261187,-5.773926,32.716582,-11.020410,2023,0.016,-0.006496,0.053181,11.671418
3,"Sborz, Josh",Fastball,95.4,-0.80,6.23,2392.0,6.1,204.0,7.56,18.72,3.942539,-138.762249,-9.112213,34.274743,-11.122984,2023,-0.022,-0.005338,0.052023,11.417350
4,"Sborz, Josh",Fastball,95.6,-0.83,6.19,2289.0,6.1,207.0,1.68,20.04,3.590410,-138.889526,-2.678258,33.103774,-8.516644,2023,0.016,-0.006376,0.053062,11.645242
5,"Sborz, Josh",Fastball,95.6,-0.66,6.25,2409.0,5.9,208.0,5.16,19.68,2.251862,-139.089664,-6.092130,32.736235,-10.023430,2023,-0.194,-0.006253,0.052938,11.618080
8,"Sborz, Josh",Fastball,96.3,-0.83,6.32,2495.0,5.9,204.0,6.36,19.68,3.084347,-139.922372,-7.687240,31.772947,-8.577354,2023,0.000,-0.006394,0.053079,11.649142


In [84]:
def calculate_mean_scores(df):
    mean_scores = df.groupby('player_name')['score'].mean().reset_index()
    mean_scores.rename(columns={'score': 'mean_scores'}, inplace=True)
    mean_scores = mean_scores.sort_values(by='mean_scores', ascending=False)
    mean_scores['mean_scores'] = mean_scores['mean_scores'].round(2)
    return mean_scores

mean_scores_fastball = calculate_mean_scores(fastball_23)
mean_scores_sinker = calculate_mean_scores(sinker_23)
mean_scores_cutter = calculate_mean_scores(cutter_23)
mean_scores_slider = calculate_mean_scores(slider_23)
mean_scores_sweeper = calculate_mean_scores(sweeper_23)
mean_scores_curveball = calculate_mean_scores(curveball_23)
mean_scores_changeup = calculate_mean_scores(changeup_23)
mean_scores_splitter = calculate_mean_scores(splitter_23)

combined_scores = mean_scores_fastball
combined_scores = combined_scores.merge(mean_scores_sinker, how = 'left', on = 'player_name', suffixes=('','_sinker'))
combined_scores = combined_scores.merge(mean_scores_cutter, how = 'left', on = 'player_name', suffixes=('','_cutter'))
combined_scores = combined_scores.merge(mean_scores_slider, how = 'left', on = 'player_name', suffixes=('','_slider'))
combined_scores = combined_scores.merge(mean_scores_sweeper, how = 'left', on = 'player_name', suffixes=('','_sweeper'))
combined_scores = combined_scores.merge(mean_scores_curveball, how = 'left', on = 'player_name', suffixes=('','_curveball'))
combined_scores = combined_scores.merge(mean_scores_changeup, how = 'left', on = 'player_name', suffixes=('','_changeup'))
combined_scores = combined_scores.merge(mean_scores_splitter, how = 'left', on = 'player_name', suffixes=('','_splitter'))



combined_scores = combined_scores.rename(columns = {
    'mean_scores': 'fastball',
    'mean_scores_sinker': 'sinker',
    'mean_scores_cutter': 'cutter',
    'mean_scores_slider': 'slider',
    'mean_scores_sweeper': 'sweeper',
    'mean_scores_curveball': 'curveball',
    'mean_scores_changeup': 'changeup',
    'mean_scores_splitter': 'splitter'
    }
)

,player_name,pitch_type,release_speed,release_pos_x,release_pos_z,release_spin_rate,release_extension,spin_axis,pfx_x,pfx_z,vx0,vy0,ax,ay,az,game_year,delta_run_exp,pred,scaled,score
706725,"Gallen, Zac",Fastball,94.2,-3.04,5.93,2371.0,6.3,207.0,6.96,20.04,8.030792,-136.678457,-9.129482,31.010717,-9.459781,2023,-0.096,-0.014204,0.060889,13.363105
637238,"Manoah, Alek",Fastball,94.2,-2.92,6.07,2509.0,6.8,213.0,4.68,21.96,7.509644,-136.710936,-6.750397,35.324478,-7.983585,2023,-0.023,-0.014154,0.060839,13.352160
629625,"Romano, Jordan",Fastball,96.6,-2.86,5.85,2348.0,7.6,201.0,3.24,19.32,8.052991,-140.258419,-5.616889,35.115619,-9.900101,2023,-0.017,-0.014028,0.060713,13.324525
373940,"Romano, Jordan",Fastball,98.0,-3.08,5.70,2369.0,7.5,209.0,3.24,18.36,8.191947,-142.410740,-5.591160,32.553347,-10.480736,2023,-0.084,-0.014003,0.060688,13.319056
7667,"Gallen, Zac",Fastball,95.6,-3.00,5.77,2457.0,6.6,208.0,2.16,18.48,7.813295,-138.930232,-4.077649,30.919233,-10.928142,2023,-0.098,-0.013995,0.060681,13.317334
706674,"Gallen, Zac",Fastball,93.6,-2.96,5.88,2399.0,6.3,210.0,7.92,23.04,7.815724,-135.613236,-10.060089,35.159230,-6.612873,2023,-0.078,-0.013952,0.060637,13.307843
556243,"Romano, Jordan",Fastball,95.0,-2.91,5.78,2433.0,7.4,210.0,6.84,19.68,7.897672,-138.010034,-9.112085,31.119256,-9.678445,2023,-0.058,-0.013894,0.060579,13.295066
160286,"Webb, Jacob",Fastball,95.1,-2.85,5.49,2535.0,6.3,236.0,5.28,20.28,7.646258,-138.042017,-7.474390,34.164061,-9.088492,2023,-0.038,-0.013878,0.060564,13.291651
8498,"Webb, Jacob",Fastball,95.4,-3.04,5.49,2574.0,6.4,218.0,7.80,20.16,7.470892,-138.686703,-10.135515,33.675057,-10.160250,2023,0.161,-0.013836,0.060521,13.282261
585443,"Romano, Jordan",Fastball,96.9,-2.83,6.02,2440.0,7.6,198.0,0.12,19.32,8.199169,-140.755107,-1.983056,32.786931,-9.621142,2023,-0.119,-0.013811,0.060496,13.276911


In [ ]:
pitch_counts = data23.groupby(['player_name', 'pitch_type']).size().unstack(fill_value=0).reset_index()

combined_data = combined_scores.merge(pitch_counts, how='left', on='player_name')
combined_data.fillna(0, inplace=True)

combined_data['weighted_fastball'] = combined_data['fastball'] * combined_data['Fastball']
combined_data['weighted_sinker'] = combined_data['sinker'] * combined_data['Sinker']
combined_data['weighted_cutter'] = combined_data['cutter'] * combined_data['Cutter']
combined_data['weighted_slider'] = combined_data['slider'] * combined_data['Slider']
combined_data['weighted_sweeper'] = combined_data['sweeper'] * combined_data['Sweeper']
combined_data['weighted_curveball'] = combined_data['curveball'] * combined_data['Curveball']
combined_data['weighted_changeup'] = combined_data['changeup'] * combined_data['Changeup']
combined_data['weighted_splitter'] = combined_data['splitter'] * combined_data['Splitter']

combined_data['total_weighted_score'] = (
    combined_data['weighted_fastball'] +
    combined_data['weighted_sinker'] +
    combined_data['weighted_cutter'] +
    combined_data['weighted_slider'] +
    combined_data['weighted_sweeper'] +
    combined_data['weighted_curveball'] +
    combined_data['weighted_changeup'] +
    combined_data['weighted_splitter']
)

combined_data['total_pitches'] = (
    combined_data['Fastball'] +
    combined_data['Sinker'] +
    combined_data['Cutter'] +
    combined_data['Slider'] +
    combined_data['Sweeper'] +
    combined_data['Curveball'] +
    combined_data['Changeup'] +
    combined_data['Splitter']
)

combined_data['overall'] = combined_data['total_weighted_score'] / combined_data['total_pitches']

combined_data['overall'] = combined_data['overall'].round(2)

combined_data = combined_data[['player_name', 'overall', 'fastball', 'sinker', 'cutter', 'slider', 'sweeper', 'curveball', 'changeup', 'splitter']]

combined_data = combined_data.sort_values(by='overall', ascending=False)

combined_data.to_csv("pitch_scores.csv", index=False)
combined_data